## Variational Quantum Eigensolver

Source: Coding with Qiskit, S2E4  
https://www.youtube.com/watch?v=Z-A6G0WVI9w

Code in the video is deprecated. The code in this notebook has been updated and works with qiskit 0.38.0


```
If we have 3 molecules,
A-A
B-B
A-B
And if the ground state energy of A-B is the lowest, then A-A will react with B-B to form A-B

The computational cost of such reactions grows exponentially (cannot simulate this classically)

We can use QCs to find the ground state

We first make an educated guess of a system's wave function, and then vary it until we get a 
minimum value of the state's energy given the system's Hamiltonian

The Hamiltonian operator gives us the total energy of a system

The VQE is a hybrid algorithm, where the quantum part computes the energy and the classical
part optimizes the variational parameter (theta)

Example use case: To compute the interatomic distance of LiH

First we need an educated guess of the wave function of the molecule
This accounts for the molecules geometry, its orbitals, and the number of electrons involved

Then we need to encode this information in the qubits of our QC
The energy is a function of some parameter that can be tuned

Educated Guess = Ansatz 
Mapping = The process of encoding the ansatz into the qubits of a QC




```

In [4]:
# !pip --version
!python3.8 -m pip install pyscf

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 908.5 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 808.1 kB/s eta 0:00:00m eta 0:00:010:00:02
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!python --version
!pip install qiskit.aqua
import numpy as np
import pylab
import copy



from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE    # NumPyMinimumEigensolver will get the energies based off of classical calculuations (benchmark test)
from qiskit.aqua.components.optimizers import SLSQP                # Classical optimizer that will help update the ansatz
from qiskit.chemistry.components.initial_states import HartreeFock # Initial ansatz
from qiskit.chemistry.components.variational_forms import UCCSD    # Helps vary the Hartree Fock guess

from qiskit.chemistry.drivers import PySCFDriver                   # To set up the molecule
from qiskit.chemistry.core import Hamiltonian, QubitMappingType    # To help with the mapping

from qiskit import Aer
qiskit.utils.algorithm_globals
qiskit.utils.QuantumInstance
qiskit.algorithms.minimum_eigen_solvers.minimum_eigen_solver.MinimumEigensolver





print("Complete!")

Python 3.9.13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 416.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 315.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.8/610.8 kB 270.6 kB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 15.5/30.8 MB 524.0 kB/s eta 0:00:30
ERROR: Wheel 'scikit-learn' located at /tmp/pip-unpack-x82aj84c/scikit_learn-1.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl is invalid.


ModuleNotFoundError: No module named 'qiskit.aqua'

In [ ]:
molecule = 'H .0 .0 -{0}; Li .0 .0 {0}' # {0} means that we are varying it along the z axis
distances = np.arange(0.5, 4.25, 0.25) # distances in Angstrom
vqe_energies = []
hf_energies = []
exact_energies = []

for i, d in enumerate(distances):
    print('step: ', i)
    
    # Set up the experiment
    driver = PySCFDriver(molecule.format(d/2), basis='sto3g')
    qmolecule = driver.run()
    
    operator = Hamiltonian(qubit_mapping=QubitMappingType.PARITY, two_qubit_reduction=True, freeze_core=True,
                          orbital_reduction=[-3,-2])
    
    '''
    PARITY - ?
    two_qubit_reduction - helps speed up the calculuation
    freeze_core - freeze the orbitals within the molecule that do not contribute to bonding
    orbital_reduction - reduce the orbitals that don't contribute
    '''
    
    qubit_op, aux_ops = operator.run(qmolecule)
    
    # exact classical result
    exact_result = NumPyMinimumEigensolver(qubit_op, aux_operators=aux_ops).run()
    exact_result = operator.process_algorith_result(exact_result)
    
    # VQE
    
    # Classical optimizer
    optimizer = SLSQP(maxiter=1000)
    initial_state = HartreeFock(operator.molecule_info['num_orbitals'], 
                                operator.molecule_info['num_particles'],
                                qubit_mapping=operator._qubit_mapping, 
                                two_qubit_reduction=operator._two_qubit_reduction)
    
    var_form = UCCSD(num_orbitals=operator.molecule_info['num_orbitals'],
                    num_particles=operator.molecule_info['num_particles'],
                    initial_state=initial_state,
                    qubit_mapping=operator._qubit_mapping,
                    two_qubit_reduction=operator._two_qubit_reduction)
    
    algo = VQE(qubit_op, var_form, optimizer, aux_operators=aux_ops)
    
    vqe_result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))
    vqe_result = operator.process_algorithm_result(vqe_result)
    
    exact_energies.append(exact_result.energy)
    vqe_energies.append(vqe_result.energy)
    hf_energies.append(vqe_result.hartree_fock_energy)
    
    
    

In [ ]:
pylab.plot(distance, hf_energies, label='Hartree-Fock')
pylab.plot(distances, vqe_energies, 'o', label='VQE')
pylab.plot(distances, exact_energies, 'x', label='Exact')
pylab.xlabel('Interatomic Distance')
pylab.ylabel('Energy')
pylab.title('LiH Ground State Energy')
pylab.legend(loc='upper right')
